<a href="https://colab.research.google.com/github/ancastillar/Projects-Advanced-Topics-in-Statistics/blob/main/Proyecto_T%C3%B3picos_E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Natalia Castilla Reyes**

**Miguel Angel Quintero**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


-----------------------------------------------------------------------

# 📑 Pronóstico de las ventas de diversos productos y evaluación de la satisfacción de los clientes de Walmart (US) mediante redes neuronales.

----------------------------------------------------------

In [2]:
#@title Instalación dependencias
!pip install -q findspark
!pip install "dask[array]"       # Install requirements for dask array
!pip install "dask[dataframe]"   # Install requirements for dask dataframe
!pip install "dask[diagnostics]" # Install requirements for dask diagnostics
!pip install "dask[distributed]" # Install requirements for distributed dask

     |████████████████████████████████| 122kB 7.4MB/s 
     |████████████████████████████████| 727kB 6.6MB/s 
ERROR: distributed 2021.6.2 has requirement dask==2021.06.2, but you'll have dask 2.12.0 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
#######################SPARK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [4]:
#@title Librerias necesarias

import time
import math
import datetime

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import dask
from dask.dataframe import DataFrame as dd
import warnings
warnings.filterwarnings("ignore")


pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']

ruta = "/content/drive/MyDrive/Proyecto_topicos/"



##1. 📌 Motivación del proyecto

Realizar correctamente el pronóstico de las ventas de ciertos productos es de gran relevancia para cierto tipo de empresas ya que así pueden tener una medición de la demanda de sus productos. Al tener una idea de la demanda de sus productos, las empresas pueden adelantarse a las necesidades de sus clientes y de esta manera tomar decisiones de negocio. Algunas de las decisiones de negocio usuales son: **asegurarse de contratar el número suficiente de empleados, tener un presupuesto aproximado de los gastos operativos y realizar el correcto restock de los productos**. Por lo anterior, realizar malos pronósticos conduce a pérdidas monetarias en la empresa, como los gastos generados al acumular un excedente de productos en épocas de baja demanda o la pérdida de beneficios
cuando hay escasez de suministros y hay una gran demanda de estos. 

Este trabajo se dividirá en dos partes:
1. Se realizará un pronostico del volumen de ventas para los distintos productos de Walmart.
2. Se evaluará la satisfacción de los clientes de Walmart mediante análisis de sentimientos de tweets.

**Esto con el objetivo de obtener una perspectiva más completa de las necesidades del cliente y permitir una mejor toma de decisiones.**

##2. Planteamiento del problema

Para realizar los pronósticos de las ventas, se emplearán los datos de la competencia de Kaggle: https://www.kaggle.com/c/m5-forecasting-accuracy/data. La tarea consiste en predecir el volumen de ventas (es decir, el número de productos vendidos) para 3.049 productos de Walmart y para cada una de sus tiendas, durante un período de 28 días. Los productos abarcan 3 categorías (Hobbies, Hogar, Alimentación) y 7 departamentos en 3 estados (CA, TX, WI). En resumen, la entrada de nuestro algoritmo es
una serie temporal del volumen de ventas de cada producto durante los últimos 5 años. Finalmente, emplearemos distintos tipos y arquitecturas de redes neuronales para resolver este problema.

Por otra parte, para evaluar la satisfacción del cliente se emplearán los tweets, que se extraeran mediante la API de https://twitter.com/. Los tweets de interés serán los relacionados con Walmart y serán seleccionados según los 7 departamentos en 3 estados de nuestra fuente de datos proporcionada por Kaggle.


##3. Pronostico del volumen de ventas

###3.1. 💾 Descripción de los datos 

**Los datos:** Los datos fueron recolectados en 3 diferentes estados de USA: California (CA), Texas (TX) y Wisconsin (WI). La información de las ventas se encuentra registrada entre Enero del 2011 y Junio del 2016.Por otra parte, los datos comprenden 3049 productos individuales de 3 categorías y 7 departamentos, vendidos en 10 tiendas de 3 estados. Por lo tanto, podemos crear diversas series temporales por ejemplo: podemos crear 1 serie temporal para todas las ventas, 3 series temporales para todas las ventas por estado, y así sucesivamente. La categoría más grande son las ventas de todos los productos individuales 3049 por 10 tiendas para 30490 series temporales.

*Los datos de entrenamiento comprenden 3 archivos:*

*  sell_prices.csv - Registra el precio de los productos vendidos por tienda y por fecha.

* calendar.csv - Contiene información sobre las fechas de venta de los productos. Incluye información para cada fecha, como los nombres de los eventos, los tipos de eventos, el día de la semana, un ID para identificar el año y la semana.

* sales_train_validation.csv  - Contiene 1 columna para cada uno de los 1941 días desde el 29-01-2011 y el 22-05-2016; sin incluir el periodo de validación de 28 días hasta el 19-06-2016. También incluye el id de cada artículo, departamento, categoría, tienda y estado. El número de filas es 30490 para todas las combinaciones de 30490 artículos y 10 tiendas




### 3.2. 🔍 Exploración de los datos

* En esta sección realizaremos el análisis detallado de las fuentes de datos que se tienen. 
* Para realizar este análisis emplearemos: Pandas y Pyspark (para algunos procesos debido a la volumetría de los datos).

### 3.3. **Datos para los pronósticos**
-------------------------------------------------------------

**Importación de fuente de datos**

In [ ]:
sales_train = pd.read_csv(ruta+ "sales_train_evaluation.csv")
sales_eval = pd.read_csv(ruta + "sales_train_validation.csv")
calendar = pd.read_csv(ruta +  "calendar.csv")
sell_prices = pd.read_csv(ruta + "sell_prices.csv")

In [ ]:
#Dimensionalidad de los datos
print("Dimensión de sales_train:", sales_train.shape)
print("Dimensión de sales_eval:", sales_eval.shape)
print("Dimensión de calenar:", calendar.shape)
print("Dimensión de sell_prices:", sales_train.shape)

Dimensión de sales_train: (30490, 1947)
Dimensión de sales_eval: (30490, 1919)
Dimensión de calenar: (1969, 14)
Dimensión de sell_prices: (30490, 1947)


*Tenemos 1947 columnas, en donde para cada producto (id) hay variables d_1- d_1913*. Por lo tanto, debemos darle un **formato adecuado a nuestro dataframe sales_train**. Para ello emplearemos la  función *melt* de pandas que nos permitirá tener una fuente de datos 30490x1941 (59181090 filas).

In [ ]:
sales_train

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,...,2,2,0,0,0,2,0,0,1,1,1,0,0,1,0,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,...,1,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,...,1,0,2,1,1,0,0,1,0,0,1,0,3,3,1,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,1,0,2,1,1,2,0,2,1,1,0,2,1,1,1,1,4,6,0,1,1,1,0


**Agregamos los datos de prueba**

In [ ]:
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales_train[col] = 0
    sales_train[col] = sales_train[col].astype(np.int16)

In [ ]:
sales_train0= pd.melt(sales_train, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 
                             'store_id', 'state_id'], var_name='d', value_name='sold').dropna()
sales_train0.to_csv(ruta+'sales_train_melt.csv', index=False)

**Visualizamos la transformación**

In [ ]:
sales_train0

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


#### **Unión de las fuentes de datos**
----------------------------------------------
* El objetivo de esta parte es contruir un único dataframe que contenga toda la información necesaria.

* Debido al tamaño de los datos sales_train luego de la transformación melt, se debe emplear pyspark para realizar algunos procesos.

**Exploración de la estructura del df Calendar**:

* La llave en común con sales_train será la variables *d*

In [ ]:
#######################SPARK
from spark import *
spark, sparkprofiling = Environment.start("Walmart")

Entorno Configurado Satisfactoriamente:
                    import pandas as pd
                    import numpy as np
                    import os
                    import subprocess
                    from matplotlib import *
                    import findspark
                    from pyspark.sql import SparkSession
                    import spark_df_profiling


In [ ]:
os.environ['JAVA_HOME'] = "usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME'] = "/content/spark-2.4.0-bin-hadoop2.7"

import findspark
findspark.init()
###################################Pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
import pyspark.sql.functions as f

spark= SparkSession \
  .builder \
  .master("local[*]")\
  .appName("Walmart") \
  .config("spark.memory.fraction", 0.8) \
  .config("spark.executor.memory", "4G") \
  .config("spark.driver.memory", "45G")\
  .config("spark.sql.shuffle.partitions" , "10") \
  .config("spark.memory.offHeap.enabled","true")\
  .config("spark.memory.offHeap.size","16g")\
  .config('spark.network.timeout',1200)\
  .config("spark.dynamicAllocation.enabled", "true")\
  .config("spark.driver.memoryOverhead", 2048) \
  .config("spark.executor.memoryOverhead", 2048)\
  .getOrCreate()

In [ ]:
df = spark.read.option("header", "true").option("encoding", "latin1").csv(ruta+"sales_train_melt.csv")
calendar = spark.read.option("header", "true").option("encoding", "latin1").csv(ruta+"calendar.csv")
sell_prices = spark.read.option("header", "true").option("encoding", "latin1").csv(ruta+"sell_prices.csv")

In [ ]:
df.show()
print("Cantidad de filas en df:", df.count())

+--------------------+-------------+---------+-------+--------+--------+---+----+
|                  id|      item_id|  dept_id| cat_id|store_id|state_id|  d|sold|
+--------------------+-------------+---------+-------+--------+--------+---+----+
|HOBBIES_1_001_CA_...|HOBBIES_1_001|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   0|
|HOBBIES_1_002_CA_...|HOBBIES_1_002|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   0|
|HOBBIES_1_003_CA_...|HOBBIES_1_003|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   0|
|HOBBIES_1_004_CA_...|HOBBIES_1_004|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   0|
|HOBBIES_1_005_CA_...|HOBBIES_1_005|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   0|
|HOBBIES_1_006_CA_...|HOBBIES_1_006|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   0|
|HOBBIES_1_007_CA_...|HOBBIES_1_007|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   0|
|HOBBIES_1_008_CA_...|HOBBIES_1_008|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|  12|
|HOBBIES_1_009_CA_...|HOBBIES_1_009|HOBBIES_1|HOBBIES|    CA_1|      CA|d_1|   2|
|HOBBIES_1_010_C

In [ ]:
%%time
df = df.join(calendar, on="d", how="left")
df.show()

+---+--------------------+-------------+---------+-------+--------+--------+----+----------+--------+--------+----+-----+----+------------+------------+------------+------------+-------+-------+-------+
|  d|                  id|      item_id|  dept_id| cat_id|store_id|state_id|sold|      date|wm_yr_wk| weekday|wday|month|year|event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|
+---+--------------------+-------------+---------+-------+--------+--------+----+----------+--------+--------+----+-----+----+------------+------------+------------+------------+-------+-------+-------+
|d_1|HOBBIES_1_001_CA_...|HOBBIES_1_001|HOBBIES_1|HOBBIES|    CA_1|      CA|   0|2011-01-29|   11101|Saturday|   1|    1|2011|        null|        null|        null|        null|      0|      0|      0|
|d_1|HOBBIES_1_002_CA_...|HOBBIES_1_002|HOBBIES_1|HOBBIES|    CA_1|      CA|   0|2011-01-29|   11101|Saturday|   1|    1|2011|        null|        null|        null|        null|      0|  

Exploración de la base: **sell_prices**

* Variables en común con sales_price: item_id, store_id, wm_yr_wk

In [ ]:
sell_prices.show()

+--------+-------------+--------+----------+
|store_id|      item_id|wm_yr_wk|sell_price|
+--------+-------------+--------+----------+
|    CA_1|HOBBIES_1_001|   11325|      9.58|
|    CA_1|HOBBIES_1_001|   11326|      9.58|
|    CA_1|HOBBIES_1_001|   11327|      8.26|
|    CA_1|HOBBIES_1_001|   11328|      8.26|
|    CA_1|HOBBIES_1_001|   11329|      8.26|
|    CA_1|HOBBIES_1_001|   11330|      8.26|
|    CA_1|HOBBIES_1_001|   11331|      8.26|
|    CA_1|HOBBIES_1_001|   11332|      8.26|
|    CA_1|HOBBIES_1_001|   11333|      8.26|
|    CA_1|HOBBIES_1_001|   11334|      8.26|
|    CA_1|HOBBIES_1_001|   11335|      8.26|
|    CA_1|HOBBIES_1_001|   11336|      8.26|
|    CA_1|HOBBIES_1_001|   11337|      8.26|
|    CA_1|HOBBIES_1_001|   11338|      8.26|
|    CA_1|HOBBIES_1_001|   11339|      8.26|
|    CA_1|HOBBIES_1_001|   11340|      8.26|
|    CA_1|HOBBIES_1_001|   11341|      8.26|
|    CA_1|HOBBIES_1_001|   11342|      8.26|
|    CA_1|HOBBIES_1_001|   11343|      8.26|
|    CA_1|

**Contrucción del dataframe total**

In [ ]:
%%time
df = df.join(sell_prices, on=['store_id','item_id','wm_yr_wk'], how='left')

df.show()

+--------+-----------+--------+-----+--------------------+-------+------+--------+----+----------+---------+----+-----+----+--------------+------------+------------+------------+-------+-------+-------+----------+
|store_id|    item_id|wm_yr_wk|    d|                  id|dept_id|cat_id|state_id|sold|      date|  weekday|wday|month|year|  event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|sell_price|
+--------+-----------+--------+-----+--------------------+-------+------+--------+----+----------+---------+----+-----+----+--------------+------------+------------+------------+-------+-------+-------+----------+
|    CA_1|FOODS_1_001|   11113| d_85|FOODS_1_001_CA_1_...|FOODS_1| FOODS|      CA|   1|2011-04-23| Saturday|   1|    4|2011|          null|        null|        null|        null|      0|      0|      0|       2.0|
|    CA_1|FOODS_1_001|   11113| d_86|FOODS_1_001_CA_1_...|FOODS_1| FOODS|      CA|   0|2011-04-24|   Sunday|   2|    4|2011|OrthodoxEaster|   Re

In [ ]:
#Guardar el resultado del proceso
%%time 
df.write.parquet(ruta+"df_total.parquet", mode='overwrite')

**PRUEBA**

Realizamos una prueba de lectura de datos con el objetivo de verificar que el dataframe total creado no haya quedado vacio.

In [ ]:
df = spark.read.option("header", "true").option("encoding", "latin1").parquet(ruta+"df_total.parquet")
print("Cantidad de datos en df:",df.count())

#### **Entendimiento de los datos (Pronóstico)**
------------------------------------------------------

In [5]:
#@title Funciones globales

#Downcast para liberar memoria

def downcast(df):
  
    """ Esta función nos permite adecuar el tipo de datos que estamos empleando de tal manera que podamos 
        liberar memoria"""

    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  


def introduce_nulls(df):
    idx = pd.date_range(df.date.dt.date.min(), df.date.dt.date.max())
    df = df.set_index('date')
    df = df.reindex(idx)
    df.reset_index(inplace=True)
    df.rename(columns={'index':'date'},inplace=True)
    return df

def plot_metric(df,state,store,metric):
    store_sales = df[(df['state_id']==state)&(df['store_id']==store)&(df['date']<='2016-05-22')]
    food_sales = store_sales[store_sales['cat_id']=='FOODS']
    store_sales = store_sales.groupby(['date','snap_'+state],as_index=False)['sold','revenue'].sum()
    snap_sales = store_sales[store_sales['snap_'+state]==1]
    non_snap_sales = store_sales[store_sales['snap_'+state]==0]
    food_sales = food_sales.groupby(['date','snap_'+state],as_index=False)['sold','revenue'].sum()
    snap_foods = food_sales[food_sales['snap_'+state]==1]
    non_snap_foods = food_sales[food_sales['snap_'+state]==0]
    non_snap_sales = introduce_nulls(non_snap_sales)
    snap_sales = introduce_nulls(snap_sales)
    non_snap_foods = introduce_nulls(non_snap_foods)
    snap_foods = introduce_nulls(snap_foods)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=non_snap_sales['date'],y=non_snap_sales[metric],
                           name='Total '+metric+'(Non-SNAP)'))
    fig.add_trace(go.Scatter(x=snap_sales['date'],y=snap_sales[metric],
                           name='Total '+metric+'(SNAP)'))
    fig.add_trace(go.Scatter(x=non_snap_foods['date'],y=non_snap_foods[metric],
                           name='Food '+metric+'(Non-SNAP)'))
    fig.add_trace(go.Scatter(x=snap_foods['date'],y=snap_foods[metric],
                           name='Food '+metric+'(SNAP)'))
    fig.update_yaxes(title_text='Total de productos vendidos' if metric=='sold' else 'Total de ingresos($)')
    fig.update_layout(template='seaborn',title=store)
    fig.update_layout(
        xaxis=dict(
        #autorange=True,
        range = ['2011-01-29','2016-05-22'],
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=2,
                     label="2y",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=4,
                     label="4y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ))
    return fig


def calmap(cal_data, state, store, scale):
    cal_data = cal_data[(cal_data['state_id']==state)&(cal_data['store_id']==store)]
    years = cal_data.year.unique().tolist()
    fig = make_subplots(rows=len(years),cols=1,shared_xaxes=True,vertical_spacing=0.005)
    r=1
    for year in years:
        data = cal_data[cal_data['year']==year]
        data = introduce_nulls(data)
        fig.add_trace(go.Heatmap(
            z=data.sold,
            x=data.week,
            y=data.day_name,
            hovertext=data.date.dt.date,
            coloraxis = "coloraxis",name=year,
        ),r,1)
        fig.update_yaxes(title_text=year,tickfont=dict(size=5),row = r,col = 1)
        r+=1
    fig.update_xaxes(range=[1,53],tickfont=dict(size=10), nticks=53)
    fig.update_layout(coloraxis = {'colorscale':scale})
    fig.update_layout(template='seaborn', title=store)
    return fig

In [6]:
df = pd.read_parquet(ruta+"df_total_aux.parquet")
print("dimensiòn del dataframe:", df.shape)

dimensiòn del dataframe: (59181090, 22)


In [7]:
df_bd = np.round(df.memory_usage().sum()/(1024*1024),1)
df['wday'] =df['wday'].astype(int)
df['month'] =df['month'].astype(int)
df['snap_CA'] =df['snap_CA'].astype(int)
df['snap_TX'] =df['snap_TX'].astype(int)
df['snap_WI'] =df['snap_WI'].astype(int)
df['sell_price'] =df['sell_price'].astype(float)
df['sold'] =df['sold'].astype(float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 22 columns):
 #   Column        Dtype  
---  ------        -----  
 0   store_id      object 
 1   item_id       object 
 2   wm_yr_wk      object 
 3   d             object 
 4   id            object 
 5   dept_id       object 
 6   cat_id        object 
 7   state_id      object 
 8   sold          float64
 9   date          object 
 10  weekday       object 
 11  wday          int64  
 12  month         int64  
 13  year          object 
 14  event_name_1  object 
 15  event_type_1  object 
 16  event_name_2  object 
 17  event_type_2  object 
 18  snap_CA       int64  
 19  snap_TX       int64  
 20  snap_WI       int64  
 21  sell_price    float64
dtypes: float64(2), int64(5), object(15)
memory usage: 9.7+ GB


In [ ]:
#Proceso para liberar memoria
%%time
df = downcast(df)
df_ad = np.round(df.memory_usage().sum()/(1024*1024),1)

dic = {'DataFrame':['DataFrame total'],
       'Antes DWNC':[df_bd],
       'Después DWNC':[df_ad]}

memory = pd.DataFrame(dic)
memory = pd.melt(memory, id_vars='DataFrame', var_name='Status', value_name='Memory (MB)')
memory.sort_values('Memory (MB)',inplace=True)
fig = px.bar(memory, x='DataFrame', y='Memory (MB)', color='Status', barmode='group', text='Memory (MB)')
fig.update_traces(texttemplate='%{text} MB', textposition='outside')
fig.update_layout(template='seaborn', title='Efecto del proceso de DWNC')
fig.show()

CPU times: user 1min 28s, sys: 1.31 s, total: 1min 29s
Wall time: 1min 28s


**Clasificación de la información**

In [ ]:
group = df.groupby(['state_id','store_id','cat_id','dept_id'],as_index=False)['item_id'].count().dropna()
group['USA'] = 'USA'
group.rename(columns={'state_id':'Estado','store_id':'Tienda','cat_id':'Categoría','dept_id':'Departamento','item_id':'Cantidad'},inplace=True)
fig = px.treemap(group, path=['USA', 'Estado', 'Tienda', 'Categoría', 'Departamento'], values='Cantidad',
                  color='Cantidad',
                  color_continuous_scale= px.colors.sequential.Sunset,
                  title='Walmart: Distribución de Items',)
fig.update_layout(template='seaborn')
fig.show(renderer="colab")

**Estudio de la distribución de los precios**

In [ ]:
group_price_store = pd.DataFrame(df.groupby(['state_id','store_id','item_id'])['sell_price'].mean().dropna()).reset_index()
fig = px.violin(group_price_store, x='store_id', color='state_id', y='sell_price',box=True, hover_name='item_id', labels={"state_id": "Estado"})
fig.update_xaxes(title_text='Tienda')
fig.update_yaxes(title_text='Precios de Venta($)')
fig.update_layout(template='seaborn',title='Distribución de los precios por tienda')

fig.show()

Notas sobre la distribución de precios por tienda:
* Se observa que la distribucion de los precios de venta respecto a las 10 tiendas es uniforme. El valor de la mediana fluctúa entre los valores 3.34 y 3.42.


**Análisis de la distribución de precios por categoría**

In [ ]:
group_price_cat = pd.DataFrame(df.groupby(['store_id','cat_id','item_id'],as_index=False)['sell_price'].mean().dropna()).reset_index()
fig = px.violin(group_price_cat, x='store_id', color='cat_id', y='sell_price',box=True, hover_name='item_id',  labels={"cat_id": "Categoría"})
fig.update_xaxes(title_text='Tienda')
fig.update_yaxes(title_text='Precio de Venta($)')
fig.update_layout(template='seaborn',title='Distribución de los precios por categoría',
                 )
fig.show()

Notas sobre el gráfico:

* Los items con categoría "FOODS" suelen ser más económicos que los items de las dos categorías restantes.

**Análisis de la cantidad de productos vendidos por tienda**

In [8]:
group = pd.DataFrame(df.groupby(['year','date','state_id','store_id'], as_index=False)['sold'].sum().dropna()).reset_index()
fig = px.violin(group, x='store_id', color='state_id', y='sold',box=True, labels={"state_id": "Estado"})
fig.update_xaxes(title_text='Tienda')
fig.update_yaxes(title_text='Total de productos vendidos')
fig.update_layout(template='seaborn',title='Distribución de productos vendidos por tienda')
fig.show()

**Análisis de los productos vendidos respecto al tiempo transcurrido**

In [ ]:
#@title code
fig = go.Figure()
title = 'Productos vendidos en cada tienda respecto al tiempo de estudio'
years = group.year.unique().tolist()
buttons = []
y=3
for state in group.state_id.unique().tolist():
    group_state = group[group['state_id']==state]
    for store in group_state.store_id.unique().tolist():
        group_state_store = group_state[group_state['store_id']==store]
        fig.add_trace(go.Scatter(name=store, x=group_state_store['date'], y=group_state_store['sold'], showlegend=True, 
                                   yaxis='y'+str(y) if y!=1 else 'y'))
    y-=1

fig.update_layout(
        xaxis=dict(
        #autorange=True,
        range = ['2011-01-29','2016-05-22'],
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=2,
                     label="2y",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=4,
                     label="4y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.33],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks="",
        title='WI',
        titlefont={"size":20},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.33, 0.66],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks="",
        title = 'TX',
        titlefont={"size":20},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.66, 1],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks='',
        title="CA",
        titlefont={"size":20},
        type="linear",
        zeroline=False
    )
    )
fig.update_layout(template='seaborn', title=title)
fig.show()

<h2 style="background-color:DodgerBlue; color:white" > Análisis por estado</h2>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#cal" role="tab" aria-controls="profile">California<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#tex" role="tab" aria-controls="messages">Texas<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#wis" role="tab" aria-controls="settings">Wisconsin<span class="badge badge-primary badge-pill">3</span></a>
<br>


En esta sección se analizará las ventas y las ganancias de cada una de las tiendas. Se realizarán tres gráficos por tienda, los detalles de cada uno de los gráficos realizados se encuentran a continuación:

* Análisis de las ventas diarias de cada tienda.

* Análisis de las ganancias diarias de cada tienda.

* Mapa de calor con las ventas diarias.

**Primero definimos la variable ingresos**

In [ ]:
df['revenue'] = df['sold']*df['sell_price'].astype(np.float32)

In [ ]:
cal_data = group.copy()
cal_data = cal_data[cal_data.date <= '22-05-2016']
cal_data['week'] = cal_data.date.dt.weekofyear
cal_data['day_name'] = cal_data.date.dt.day_name()

<a id="cal" class="btn btn-primary btn-lg btn-block active" role="button" aria-pressed="true" style="color:white;">California</a>
<a id='csn'><span class="label label-info">Tiendas ubicadas en california:</span></a>
<nav aria-label="Store Navigator">
  <ul class="pagination">
    <li class="page-item"><a class="page-link" href='#C1'>CA_1</a></li>
    <li class="page-item"><a class="page-link" href="#C2">CA_2</a></li>
    <li class="page-item"><a class="page-link" href="#C3">CA_3</a></li>
    <li class="page-item"><a class="page-link" href="#C4">CA_4</a></li>
  </ul>
</nav>

In [ ]:
fig = plot_metric(df,'CA','CA_1','sold')
fig.show()

## 4. 🛠️ Datos Twitter  (En construcción)

Extraer en tiempo real los datos de twitter, con las especificaciones requeridas (tema y ubicación), y crear una base de datos que vaya almacenando estos tweets para luego procesarlos.

Para entrenar la red neuronal que extraerá las emociones del texto de los tweets se implentará las siguientes bases de datos extraídas de https://archive.ics.uci.edu/ml/index.php.

### 4.1. **Datos para la extracción de emociones**
--------------------------------------------------------

In [ ]:
data = pd.read_csv(ruta+"text_emotion.csv")
data0=pd.read_csv(ruta+"train.txt", sep=';',names=["content","sentiment"])
print("Dimension de df:", data.shape)
print("Dimension de df0:", data0.shape)

Dimension de df: (40000, 4)
Dimension de df0: (16000, 2)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   author     40000 non-null  object
 3   content    40000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


Para la base de datos "text_emotions", se tiene:


In [ ]:
data.head(5)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [ ]:
data0.head(5)

,content,sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


##### 4.1.1. **Preparación de los datos de entrenamiento**

In [ ]:
data=data[['content','sentiment']]
data0=data0[['content','sentiment']]
data=pd.concat([data, data0],axis=0)
data.head(4)

,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm


**Creación del diccionario para corregir posibles errores gramaticales en las frases**

In [ ]:
misspell_data = pd.read_csv(ruta+ "spell.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

#Muestra del diccionario
{v:miss_corr[v] for v in [list(miss_corr.keys())[k] for k in range(20)]}

{'Steffen': 'Stephen',
 'abilitey': 'ability',
 'abouy': 'about',
 'absorbtion': 'absorption',
 'accidently': 'accidentally',
 'accomodate': 'accommodate',
 'acommadate': 'accommodate',
 'acord': 'accord',
 'adultry': 'adultery',
 'aggresive': 'aggressive',
 'alchohol': 'alcohol',
 'alchoholic': 'alcoholic',
 'allieve': 'alive',
 'alright': 'all_right',
 'aquantance': 'acquaintance',
 'equire': 'acquire',
 'nevade': 'Nevada',
 'presbyterian': 'Presbyterian',
 'rsx': 'RSX',
 'susan': 'Susan'}

**Datos para eliminar las contracciones de las frases**

In [ ]:
#Contracciones

contractions = pd.read_csv(ruta+"datasets_697638_1220322_contractions.csv")
cont_dic = dict(zip(contractions.Contraction, contractions.Meaning))

#Muestra

{v:cont_dic[v] for v in [list(cont_dic.keys())[k] for k in range(10)]}

{"'aight": 'alright',
 "'cause": 'because',
 "ain't": 'is not',
 "amn't": 'am not',
 "aren't": 'are not',
 "can't": 'cannot',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "daren't": 'dare not'}

**¿Qué tipo de frases tenemos en nuestra base de datos?**

In [ ]:
fig = px.bar(pd.DataFrame(data.sentiment.value_counts()).reset_index(), x="index", y="sentiment", 
             labels={'sentiment':'Cantidad de Frases', 'index':'sentimiento'})
fig.update_layout(template='seaborn',title='Distribución de las frases por sentimiento')
fig.show()